<center><a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>

# Assessment

Congratulations on going through today's course! Hopefully, you've learned some valuable skills along the way and had fun doing it. Now it's time to put those skills to the test. In this assessment, you will train a new model that is able to recognize fresh and rotten fruit. You will need to get the model to a validation accuracy of `92%` in order to pass the assessment, though we challenge you to do even better if you can. You will have the use the skills that you learned in the previous exercises. Specifically, we suggest using some combination of transfer learning, data augmentation, and fine tuning. Once you have trained the model to be at least 92% accurate on the validation dataset, save your model, and then assess its accuracy. Let's get started! 

## The Dataset

In this exercise, you will train a model to recognize fresh and rotten fruits. The dataset comes from [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification), a great place to go if you're interested in starting a project after this class. The dataset structure is in the `data/fruits` folder. There are 6 categories of fruits: fresh apples, fresh oranges, fresh bananas, rotten apples, rotten oranges, and rotten bananas. This will mean that your model will require an output layer of 6 neurons to do the categorization successfully. You'll also need to compile the model with `categorical_crossentropy`, as we have more than two categories.

<img src="./images/fruits.png" style="width: 600px;">

## Load ImageNet Base Model

We encourage you to start with a model pretrained on ImageNet. Load the model with the correct weights, set an input shape, and choose to remove the last layers of the model. Remember that images have three dimensions: a height, and width, and a number of channels. Because these pictures are in color, there will be three channels for red, green, and blue. We've filled in the input shape for you. This cannot be changed or the assessment will fail. If you need a reference for setting up the pretrained model, please take a look at [notebook 05b](05b_presidential_doggy_door.ipynb) where we implemented transfer learning.

In [4]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights="imagenet",
    input_shape=(224, 224, 3),
    include_top=False)

58892288/58889256 [==============================] - 0s 0us/step


## Freeze Base Model

Next, we suggest freezing the base model, as done in [notebook 05b](05b_presidential_doggy_door.ipynb). This is done so that all the learning from the ImageNet dataset does not get destroyed in the initial training.

In [5]:
# Freeze base model
base_model.trainable = False

## Add Layers to Model

Now it's time to add layers to the pretrained model. [Notebook 05b](05b_presidential_doggy_door.ipynb) can be used as a guide. Pay close attention to the last dense layer and make sure it has the correct number of neurons to classify the different types of fruit.

In [7]:
# Create inputs with correct shape
inputs = base_model.input

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.Flatten()(base_model.output)


# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs=inputs, outputs=outputs)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

## Compile Model

Now it's time to compile the model with loss and metrics options. Remember that we're training on a number of different categories, rather than a binary classification problem.

In [9]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'], optimizer='adam')

## Augment the Data

If you'd like, try to augment the data to improve the dataset. Feel free to look at [notebook 04a](04a_asl_augmentation.ipynb) and [notebook 05b](05b_presidential_doggy_door.ipynb) for augmentation examples. There is also documentation for the [Keras ImageDataGenerator class](https://keras.io/api/preprocessing/image/#imagedatagenerator-class). This step is optional, but it may be helpful to get to 92% accuracy.

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator( rotation_range=20,
  width_shift_range=0.1,
  height_shift_range=0.1,
  shear_range=0.1,
  zoom_range=0.2,
  horizontal_flip=True,
  vertical_flip=True,
  rescale=1./255,  
  preprocessing_function=keras.applications.vgg16.preprocess_input)

## Load Dataset

Now it's time to load the train and validation datasets. Pick the right folders, as well as the right `target_size` of the images (it needs to match the height and width input of the model you've created). For a reference, check out [notebook 05b](05b_presidential_doggy_door.ipynb).

In [13]:
train_path='data/fruits/train/'  # From Kaggle
test_path='data/fruits/valid/'

train_it = datagen.flow_from_directory(train_path, 
                                       target_size=[224,224], 
                                       color_mode='rgb', 
                                       class_mode="categorical",
                                       batch_size = 32)
# load and iterate test dataset
test_it = datagen.flow_from_directory(test_path, 
                                      target_size=[224,224], 
                                      color_mode='rgb', 
                                      class_mode="categorical",
                                    batch_size = 32)


Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


## Train the Model

Time to train the model! Pass the `train` and `valid` iterators into the `fit` function, as well as setting the desired number of epochs.

In [14]:
model.fit(train_it,
          validation_data=test_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=test_it.samples/test_it.batch_size,
          epochs=10)

Epoch 1/10
37/36 [==============================] - 29s 780ms/step - loss: 1.0865 - accuracy: 0.6227 - val_loss: 0.4316 - val_accuracy: 0.8389
Epoch 2/10
37/36 [==============================] - 21s 557ms/step - loss: 0.3068 - accuracy: 0.8968 - val_loss: 0.2384 - val_accuracy: 0.9271
Epoch 3/10
37/36 [==============================] - 20s 533ms/step - loss: 0.2129 - accuracy: 0.9340 - val_loss: 0.2514 - val_accuracy: 0.9119
Epoch 4/10
37/36 [==============================] - 20s 534ms/step - loss: 0.1731 - accuracy: 0.9450 - val_loss: 0.2153 - val_accuracy: 0.9271
Epoch 5/10
37/36 [==============================] - 20s 541ms/step - loss: 0.1468 - accuracy: 0.9492 - val_loss: 0.1952 - val_accuracy: 0.9392
Epoch 6/10
37/36 [==============================] - 20s 534ms/step - loss: 0.1150 - accuracy: 0.9602 - val_loss: 0.1507 - val_accuracy: 0.9362
Epoch 7/10
37/36 [==============================] - 20s 531ms/step - loss: 0.1238 - accuracy: 0.9611 - val_loss: 0.1986 - val_accuracy: 0.9422

## Unfreeze Model for Fine Tuning

If you have reached 92% validation accuracy already, this next step is optional. If not, we suggest fine tuning the model with a very low learning rate.

In [16]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.Adam(learning_rate = .00001),
              loss =keras.losses.BinaryCrossentropy(from_logits=True) , metrics =[keras.metrics.BinaryAccuracy()])

In [17]:
model.fit(train_it,
          validation_data=test_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=test_it.samples/test_it.batch_size,
          epochs=10)

Epoch 1/10
37/36 [==============================] - 34s 915ms/step - loss: 0.6360 - binary_accuracy: 0.9904 - val_loss: 0.6376 - val_binary_accuracy: 0.9858
Epoch 2/10
37/36 [==============================] - 21s 574ms/step - loss: 0.6335 - binary_accuracy: 0.9935 - val_loss: 0.6381 - val_binary_accuracy: 0.9853
Epoch 3/10
37/36 [==============================] - 20s 553ms/step - loss: 0.6321 - binary_accuracy: 0.9968 - val_loss: 0.6384 - val_binary_accuracy: 0.9838
Epoch 4/10
37/36 [==============================] - 20s 554ms/step - loss: 0.6339 - binary_accuracy: 0.9929 - val_loss: 0.6372 - val_binary_accuracy: 0.9878
Epoch 5/10
37/36 [==============================] - 21s 562ms/step - loss: 0.6320 - binary_accuracy: 0.9966 - val_loss: 0.6347 - val_binary_accuracy: 0.9909
Epoch 6/10
37/36 [==============================] - 20s 543ms/step - loss: 0.6312 - binary_accuracy: 0.9983 - val_loss: 0.6371 - val_binary_accuracy: 0.9868
Epoch 7/10
37/36 [==============================] - 21s 55

## Evaluate the Model

Hopefully, you now have a model that has a validation accuracy of 92% or higher. If not, you may want to go back and either run more epochs of training, or adjust your data augmentation. 

Once you are satisfied with the validation accuracy, evaluate the model by executing the following cell. The evaluate function will return a tuple, where the first value is your loss, and the second value is your accuracy. To pass, the model will need have an accuracy value of `92% or higher`. 

In [18]:
model.evaluate(test_it, steps=test_it.samples/test_it.batch_size)

11/10 [================================] - 4s 358ms/step - loss: 0.6349 - binary_accuracy: 0.9889


[0.6348861455917358, 0.9888551235198975]

## Run the Assessment

To assess your model run the following two cells.

**NOTE:** `run_assessment` assumes your model is named `model` and your validation data iterator is called `valid_it`. If for any reason you have modified these variable names, please update the names of the arguments passed to `run_assessment`.

In [19]:
from run_assessment import run_assessment

In [20]:
run_assessment(model, test_it)

Evaluating model 5 times to obtain average accuracy...

11/10 [================================] - 4s 353ms/step - loss: 0.6335 - binary_accuracy: 0.9939

Accuracy required to pass the assessment is 0.92 or greater.
Your average accuracy is 0.9931.

Congratulations! You passed the assessment!
See instructions below to generate a certificate.


## Generate a Certificate

If you passed the assessment, please return to the course page (shown below) and click the "ASSESS TASK" button, which will generate your certificate for the course.

<img src="./images/assess_task.png" style="width: 800px;">

<center><a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>